#### IMPORT LIBRARIES

In [23]:
from os import getenv
import configparser
import requests
import pandas as pd
import datetime
import psycopg2
from dotenv import load_dotenv
load_dotenv()

True

#### CONFIGURATIONS

In [24]:
try:
  API_KEY = getenv('API_KEY')
  LAT = getenv('LAT')
  LON = getenv('LON')
  UNITS = getenv('UNITS')

  # read database password
  DATABASE_HOST = getenv('DATABASE_HOST')
  DATABASE_USER = getenv('DATABASE_USER')
  DATABASE_NAME = getenv('DATABASE_NAME')
  DATABASE_PASSWORD = getenv('DATABASE_PASSWORD')
  
except:
  print("An exception occurred in Configurations")

#### CONNECT DATABASE

In [25]:
def dbConnect():
  try:
    conn = psycopg2.connect(f"host={DATABASE_HOST} dbname={DATABASE_NAME} user={DATABASE_USER} password={DATABASE_PASSWORD}")
    return conn
  
  except:
    print("Error occurred while connecting to Database")

## CODE

#### Parsing Raw Data Function

In [26]:
def parsingRawData(rawData: dict) -> dict[dict]:
  myDict = {
    "date": datetime.datetime.fromtimestamp(rawData["dt"]),
    "city_name": rawData["name"],
    "city_id": rawData["id"],
    "country": rawData["sys"]["country"],
    "lon": rawData["coord"]["lon"],
    "lat": rawData["coord"]["lat"],
    "base": rawData["base"],
    "temp": rawData["main"]["temp"],
    "feels_like": rawData["main"]["feels_like"],
    "timezone": rawData["timezone"]
  }
  return myDict

#### Insert Data Function

In [27]:
def insertData(parsedData: dict):
    conn = dbConnect()
    try:
        cursor = conn.cursor()

        SQL = "INSERT INTO tbldata (city_id, lat, lon, temp, feels_like, city_name, country, timezone, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
        data = (
            parsedData["city_id"],
            parsedData['lat'],
            parsedData["lon"],
            parsedData["temp"],
            parsedData["feels_like"],
            parsedData["city_name"],
            parsedData["country"],
            parsedData["timezone"],
            parsedData["date"],
            )
        cursor.execute(SQL, data)

        conn.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into tbldata")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into tbldata", error)

    finally:
        # closing database connection.
        if conn:
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")

#### API Call

In [28]:
rawData = requests.get(f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units={UNITS}").json()
rawData

{'coord': {'lon': 7.5906, 'lat': 50.3681},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10n'}],
 'base': 'stations',
 'main': {'temp': 12.84,
  'feels_like': 12.61,
  'temp_min': 11.92,
  'temp_max': 13.63,
  'pressure': 993,
  'humidity': 93},
 'visibility': 10000,
 'wind': {'speed': 15.65, 'deg': 22},
 'rain': {'1h': 0.21},
 'clouds': {'all': 100},
 'dt': 1698694920,
 'sys': {'type': 2,
  'id': 19193,
  'country': 'DE',
  'sunrise': 1698646557,
  'sunset': 1698682230},
 'timezone': 3600,
 'id': 2874772,
 'name': 'Lützel',
 'cod': 200}

In [29]:
parsedData = parsingRawData(rawData)
parsedData

{'date': datetime.datetime(2023, 10, 30, 20, 42),
 'city_name': 'Lützel',
 'city_id': 2874772,
 'country': 'DE',
 'lon': 7.5906,
 'lat': 50.3681,
 'base': 'stations',
 'temp': 12.84,
 'feels_like': 12.61,
 'timezone': 3600}

In [30]:
insertData(parsedData)

1 Record inserted successfully into tbldata
PostgreSQL connection is closed
